In [11]:
import pandas as pd
import os
import math
import numpy as np

In [12]:
# Set the working directory to 'this.dir()'
os.chdir('C:/Users/6997619/Documents/GitHub projects/DHZW-simulation_Sim-2APL/src/main/resources/DHZW_full')

# Read synthetic activities
df = pd.read_csv("DHZW_activities_locations.csv")

In [13]:
len(df)
df

,pid,hh_ID,activity_number,activity_type,day_of_week,start_time_seconds,duration_seconds,in_DHZW,postcode,coordinate_y,coordinate_x
0,1,42329,1,home,1,0,86340,1,2544E,52.046483,4.250288
1,1,42329,2,home,2,86400,86340,1,2544E,52.046483,4.250288
2,1,42329,3,home,3,172800,86340,1,2544E,52.046483,4.250288
3,1,42329,4,home,4,259200,86340,1,2544E,52.046483,4.250288
4,1,42329,5,home,5,345600,40500,1,2544E,52.046483,4.250288
...,...,...,...,...,...,...,...,...,...,...,...
1461597,84880,23178,7,shopping,4,299700,3600,0,2511,52.078513,4.317539
1461598,84880,23178,8,home,4,304200,41340,1,2532E,52.047579,4.295087
1461599,84880,23178,9,home,5,345600,86340,1,2532E,52.047579,4.295087
1461600,84880,23178,10,home,6,432000,86340,1,2532E,52.047579,4.295087


In [14]:
# Transfrom the activity type in upper case like ODiN
df['activity_type'] = df['activity_type'].str.upper()

In [15]:
# Sort the dataframe by 'Agent' and 'Day'
df = df.sort_values(['pid', 'day_of_week', 'activity_number']).reset_index(drop=True)

In [16]:
# Within each agent and day, add the previous activity and postcode visited
df['previous_activity'] = df.groupby(['pid', 'day_of_week'])['activity_type'].shift()
df['previous_postcode'] = df.groupby(['pid', 'day_of_week'])['postcode'].shift()

In [17]:
# Extract trips by selecting only activities with a previous activity within the day
df = df.dropna(subset=['previous_activity'])
len(df)

867442

In [18]:
# Select columns
df = df[['pid', 'day_of_week', 'previous_activity', 'activity_type', 'previous_postcode', 'postcode']]

In [19]:
# Transfrom PC5 to PC4

df['previous_postcode'] = df['previous_postcode'].apply(lambda x: x[:-1] if len(x) == 5 else x)
print(df['previous_postcode'].str.len().unique())

df['postcode'] = df['postcode'].apply(lambda x: x[:-1] if len(x) == 5 else x)
print(df['postcode'].str.len().unique())

[4]
[4]


### Add postcode home from synthetic population

In [20]:
# Read synthetic population to add home postcode
df_synthetic_population = pd.read_csv("DHZW_synthetic_population.csv")

# select useful coloumns
df_synthetic_population = df_synthetic_population[['pid', 'PC6']]

# Transfrom home PC6 to PC4
df_synthetic_population['PC6'] = df_synthetic_population['PC6'].str[:-2]

# Rename coloumn to match ODiN
df_synthetic_population = df_synthetic_population.rename(columns={'PC6': 'hh_PC4'})

In [21]:
# Add home PC4 to the trips
df = pd.merge(df, df_synthetic_population, on='pid')

In [22]:
# Rename like ODiN
df = df.rename(columns={'pid': 'agent_ID',
                        'day_of_week': 'day_of_week',
                        'activity_type': 'disp_activity',
                        'postcode': 'disp_arrival_PC4',
                        'previous_postcode': 'disp_start_PC4'})

In [23]:
len(df)

867442

In [24]:
# Save trips
os.chdir('C:/Users/6997619/Documents/GitHub projects/DHZW_assign_location/data/processed/analysis/ODiN')
df.to_csv('synthetic_trips.csv', index=False, header=True)